In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import skimage as sk
from skimage import measure
import os
import tifffile
from tqdm import tqdm

In [2]:
dots_data = pd.read_csv("/mnt/data/Imaging/202105-Deconwolf/data_210726/dots_data.clean.tsv.gz", sep="\t")

In [3]:
dots_data2 = dots_data.loc["20x" == dots_data["magnification"], :]

In [4]:
thresholds_table = pd.read_csv("../../data/magnifications_matching/intensity_thresholds.by_field.tsv", sep="\t")

In [5]:
ref_raw = dots_data2.loc["raw" == dots_data2["image_type"], :].reset_index(drop=True)
ref__dw = dots_data2.loc["dw" == dots_data2["image_type"], :].reset_index(drop=True)
raw_image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/20x_raw"
dw__image_folder_path = "/mnt/data/Imaging/202105-Deconwolf/data_210726/20x_dw"
mask_folder_path = "../../data/20x_mask/dilated_labels_watershed_from60x"
field_list = np.unique(dots_data2["sid"].values).tolist()

In [6]:
for current_field_id in field_list:
    print(f"Field #{current_field_id}")
    thresholds = thresholds_table.loc[current_field_id == thresholds_table["sid"], :]
    raw_thr = thresholds.loc["raw" == thresholds["image_type"], "thr"].values[0]
    dw__thr = thresholds.loc["dw" == thresholds["image_type"], "thr"].values[0]

    raw_max_z_proj = tifffile.imread(os.path.join(raw_image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    dw__max_z_proj = tifffile.imread(os.path.join(dw__image_folder_path, f"a647_{current_field_id:03d}.tif")).max(0)
    
    labels = tifffile.imread(os.path.join(mask_folder_path, f"a647_{current_field_id:03d}.dilated_labels.from_60x.tiff")).reshape(raw_max_z_proj.shape)

    field_raw_dots = ref_raw.loc[ref_raw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    field_dw__dots = ref__dw.loc[ref__dw["sid"] == current_field_id, :].sort_values("Value2", ascending=False)
    
    selected_raw_dots = field_raw_dots.loc[field_raw_dots["Value2"] >= raw_thr, :].reset_index(drop=True)
    selected_dw__dots = field_dw__dots.loc[field_dw__dots["Value2"] >= dw__thr, :].reset_index(drop=True)

    fig3, ax = plt.subplots(figsize=(20, 10), ncols=2, constrained_layout=True)
    fig3.suptitle(f"Field #{current_field_id}")
    print(" > Plotting raw")
    ax[0].set_title(f"20x_raw (n.dots={selected_raw_dots.shape[0]})")
    ax[0].imshow(
        raw_max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=raw_max_z_proj.min(), vmax=raw_max_z_proj.max(),
        resample=False, filternorm=False)
    ax[0].scatter(
        x=selected_raw_dots["y"].values,
        y=selected_raw_dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting dw")
    ax[1].set_title(f"20x_dw (n.dots={selected_dw__dots.shape[0]})")
    ax[1].imshow(
        dw__max_z_proj, cmap=plt.get_cmap("gray"), interpolation="none",
        vmin=dw__max_z_proj.min()*1.5, vmax=dw__max_z_proj.max()*.5,
        resample=False, filternorm=False)
    ax[1].scatter(
        x=selected_dw__dots["y"].values,
        y=selected_dw__dots["x"].values,
        s=30, facecolors='none', edgecolors='r', linewidth=.5)
    print(" > Plotting contours")
    for lid in tqdm(range(1, labels.max()), desc="nucleus"):
        contours = measure.find_contours(labels == lid, 0.8)
        for contour in contours:
            ax[0].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
            ax[1].scatter(x=contour[:,1], y=contour[:,0], c="yellow", s=.005)
    plt.close(fig3)
    #print(" > Exporting")
    fig3.savefig(f"../../data/magnifications_matching/overlays.field_thr/overlay_{current_field_id:03d}.png", bbox_inches='tight')
    print(" ! DONE")

Field #1


nucleus:   4%|▍         | 7/180 [00:00<00:02, 58.12it/s]

 > Plotting raw
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 180/180 [00:03<00:00, 51.52it/s]


 ! DONE
Field #2


...

 ! DONE
Field #7


nucleus:   4%|▎         | 8/221 [00:00<00:02, 73.72it/s]

 > Plotting raw
 > Plotting dw
 > Plotting contours


nucleus: 100%|██████████| 221/221 [00:04<00:00, 44.40it/s]


 ! DONE
